In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
from joblib import load
from qiskit.circuit.library import ZFeatureMap
import qiskit_machine_learning
from qiskit_machine_learning.kernels import FidelityStatevectorKernel
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.algorithms import QSVC
from sklearn.svm import SVC
import os
from sklearn.utils import resample
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [34]:
dict_data = r'C:/Qe-CaDD/Project/Stage_4_QSVM_toxicity_models/pathways_pca_16_1024/'

for files in os.listdir(dict_data):
    print(files)
    name, ext = files.split(".")
    df = pd.read_csv(rf'C:/Qe-CaDD/Project/Stage_4_QSVM_toxicity_models/pathways_pca_16_1024/{files}')

    X = df.iloc[:, 1:17].values
    y = df.iloc[:, 17:].values.squeeze()


    df_combined = pd.DataFrame(X)
    df_combined['label'] = y


    df_0 = df_combined[df_combined['label'] == 0]
    df_1 = df_combined[df_combined['label'] == 1]

    df_0_resampled = resample(df_0, replace=False, n_samples=445 // 2, random_state=42)
    df_1_resampled = resample(df_1, replace=False, n_samples=445 // 2, random_state=42)

    df_balanced = pd.concat([df_0_resampled, df_1_resampled])
    X_balanced = df_balanced.iloc[:, :-1].values
    y_balanced = df_balanced['label'].values

    x_train, x_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)
    print(f"Reduced x_train shape: {x_train.shape}")

    num_features = x_train.shape[1]
    feature_map = ZFeatureMap(feature_dimension= num_features, reps = 2, insert_barriers=  True)

    sampler = Sampler()
    fidelity = ComputeUncompute(sampler = sampler)

    kernel = FidelityStatevectorKernel(feature_map= feature_map)


    qsvc = QSVC(quantum_kernel=kernel, C=0.75 , gamma = 0.001, cache_size=10000)
    qsvc.fit(x_train, y_train)


    qsvc_predict = qsvc.predict(x_test)

    train_accuracy_qsvc = qsvc.score(x_train, y_train)
    print(f"Training Accuracy (QSVC): {train_accuracy_qsvc:.2f}")

    test_accuracy_qsvc = qsvc.score(x_test, y_test)
    print(f"Test Accuracy (QSVC): {test_accuracy_qsvc:.2f}")


    print(classification_report(y_test, qsvc_predict))


    svm = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)
    svm.fit(x_train, y_train)

 
    svm_predict = svm.predict(x_test)

    train_accuracy_svm = svm.score(x_train, y_train)
    print(f"Training Accuracy (SVM): {train_accuracy_svm:.2f}")
    
    test_score_svm = svm.score(x_test, y_test)
    print(f"Test Accuracy (SVM): {test_score_svm:.2f}")

    print(classification_report(y_test, svm_predict))

    





    #joblib.dump(svm, rf'{name}_SVM.joblib')


tox21_pca_NR-AhR.csv
Reduced x_train shape: (355, 16)


C:\Users\lohit\AppData\Local\Temp\ipykernel_16156\850499729.py:32: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


Training Accuracy (QSVC): 0.97
Test Accuracy (QSVC): 0.58
              precision    recall  f1-score   support

         0.0       0.59      0.65      0.62        46
         1.0       0.58      0.51      0.54        43

    accuracy                           0.58        89
   macro avg       0.58      0.58      0.58        89
weighted avg       0.58      0.58      0.58        89

Training Accuracy (SVM): 0.78
Test Accuracy (SVM): 0.57
              precision    recall  f1-score   support

         0.0       0.60      0.54      0.57        46
         1.0       0.55      0.60      0.58        43

    accuracy                           0.57        89
   macro avg       0.57      0.57      0.57        89
weighted avg       0.57      0.57      0.57        89

tox21_pca_NR-AR-LBD.csv
Reduced x_train shape: (355, 16)


C:\Users\lohit\AppData\Local\Temp\ipykernel_16156\850499729.py:32: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


KeyboardInterrupt: 